# Import Libraries

In [ ]:
import pandas as pd, numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Load Dataset

In [ ]:
dataset = pd.read_csv('/content/gdrive/My Drive/DigitalSkola/Dataset/keywords.csv')
dataset.head()

,Date,Market,Keyword,Average.Position,Clicks,Impressions,Cost
0,20120524,US-Market,secure online back up,0.0,0.00,0.0,0.00
1,20120524,US-Market,agile management software,1.0,21.22,260.0,25.45
2,20120524,US-Market,crm for financial,0.0,0.00,0.0,0.00
3,20120524,US-Market,disaster recovery planning for it,0.0,0.00,0.0,0.00
4,20120524,US-Market,tracking a vehicle,0.0,0.00,0.0,0.00


# Feature Engineering

In [ ]:
dataset.isnull().sum()

Date                0
Market              0
Keyword             0
Average.Position    0
Clicks              0
Impressions         0
Cost                0
dtype: int64

In [ ]:
dataset.duplicated().sum()

0

In [ ]:
dataset = dataset.drop_duplicates()

In [ ]:
dataset.shape

(346642, 7)

In [ ]:
dataset['CPI'] = dataset['Cost'] / dataset['Impressions']
dataset['CPC'] = dataset['Cost'] / dataset['Clicks']
dataset['CTR'] = dataset['Clicks'] / dataset['Impressions']
dataset.head()

In [ ]:
dataset.fillna(value=0, inplace=True)
dataset.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,Date,Market,Keyword,Average.Position,Clicks,Impressions,Cost,CPI,CPC,CTR
0,20120524,US-Market,secure online back up,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000
1,20120524,US-Market,agile management software,1.0,21.22,260.0,25.45,0.097885,1.19934,0.081615
2,20120524,US-Market,crm for financial,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000
3,20120524,US-Market,disaster recovery planning for it,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000
4,20120524,US-Market,tracking a vehicle,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000


In [ ]:
dataset.describe()

,Date,Average.Position,Clicks,Impressions,Cost,CPI,CPC,CTR
count,3.466420e+05,346642.000000,346642.000000,3.466420e+05,3.466420e+05,346642.000000,346642.000000,346642.000000
mean,2.012270e+07,0.622315,447.037645,1.908227e+04,3.663141e+03,0.276195,13.715180,0.016646
std,3.585030e+03,0.644443,7628.491960,2.997886e+05,5.770403e+04,0.651221,34.804560,0.020465
min,2.012052e+07,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
25%,2.012090e+07,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
50%,2.012102e+07,1.000000,2.430000,1.390000e+02,1.001000e+01,0.021100,1.210814,0.008233
75%,2.012122e+07,1.000000,37.037500,1.641000e+03,3.967175e+02,0.302225,12.178391,0.028483
max,2.013021e+07,12.000000,441963.500000,1.566667e+07,3.269108e+06,33.932771,865.000000,0.239375


In [ ]:
# Change inf data into 0
dataset = dataset.replace([np.inf, -np.inf], 0)

# Advanced Label Encoder

In [ ]:
dataset.Market.unique()

array(['US-Market', 'UK-Market'], dtype=object)

In [ ]:
dataset.head()

,Date,Market,Keyword,Average.Position,Clicks,Impressions,Cost,CPI,CPC,CTR
0,20120524,US-Market,secure online back up,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000
1,20120524,US-Market,agile management software,1.0,21.22,260.0,25.45,0.097885,1.19934,0.081615
2,20120524,US-Market,crm for financial,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000
3,20120524,US-Market,disaster recovery planning for it,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000
4,20120524,US-Market,tracking a vehicle,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000


In [ ]:
market_data = dataset.pivot_table(index='Market',
                                  values=['Clicks', 'Impressions', 'Cost', 'CPI', 'CPC', 'CTR'],
                                  aggfunc=[np.mean, np.median, np.max])
market_data

mean                      ...       amax                        
                 CPC       CPI       CTR  ...     Clicks        Cost Impressions
Market                                    ...                                   
UK-Market  12.027957  0.262945  0.016310  ...  156341.05   918862.86   9571421.0
US-Market  14.647569  0.283518  0.016832  ...  441963.50  3269107.70  15666669.0

[2 rows x 18 columns]

In [ ]:
mean_market = market_data['mean'].reset_index()
median_market = market_data['median'].reset_index()
max_market = market_data['amax'].reset_index()

In [ ]:
mean_market.columns = ['Market', 'mean_market_CPC', 'mean_market_CPI', 'mean_market_CTR', 'mean_market_Clicks', 'mean_market_Cost', 'mean_market_Impressions']
median_market.columns = ['Market', 'median_market_CPC', 'median_market_CPI', 'median_market_CTR', 'median_market_Clicks', 'median_market_Cost', 'median_market_Impressions']
max_market.columns = ['Market', 'max_market_CPC', 'max_market_CPI', 'max_market_CTR', 'max_market_Clicks', 'max_market_Cost', 'max_market_Impressions']

In [ ]:
market = pd.merge(mean_market, median_market, how='inner', on='Market')
market = pd.merge(market, max_market, how='inner', on='Market')
market

,Market,mean_market_CPC,mean_market_CPI,mean_market_CTR,mean_market_Clicks,mean_market_Cost,mean_market_Impressions,median_market_CPC,median_market_CPI,median_market_CTR,median_market_Clicks,median_market_Cost,median_market_Impressions,max_market_CPC,max_market_CPI,max_market_CTR,max_market_Clicks,max_market_Cost,max_market_Impressions
0,UK-Market,12.027957,0.262945,0.016310,101.473886,1320.571699,3412.243842,0.500000,0.008506,0.004620,0.28,0.63,22.0,756.5,29.065157,0.239375,156341.05,918862.86,9571421.0
1,US-Market,14.647569,0.283518,0.016832,638.002163,4957.685439,27741.795385,1.623074,0.025146,0.009835,6.46,28.57,336.0,865.0,33.932771,0.171930,441963.50,3269107.70,15666669.0


In [ ]:
del mean_market, median_market, max_market, market_data

In [ ]:
dataset = pd.merge(dataset, market, how='left', on='Market')
dataset.head()

,Date,Market,Keyword,Average.Position,Clicks,Impressions,Cost,CPI,CPC,CTR,mean_market_CPC,mean_market_CPI,mean_market_CTR,mean_market_Clicks,mean_market_Cost,mean_market_Impressions,median_market_CPC,median_market_CPI,median_market_CTR,median_market_Clicks,median_market_Cost,median_market_Impressions,max_market_CPC,max_market_CPI,max_market_CTR,max_market_Clicks,max_market_Cost,max_market_Impressions
0,20120524,US-Market,secure online back up,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000,14.647569,0.283518,0.016832,638.002163,4957.685439,27741.795385,1.623074,0.025146,0.009835,6.46,28.57,336.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0
1,20120524,US-Market,agile management software,1.0,21.22,260.0,25.45,0.097885,1.19934,0.081615,14.647569,0.283518,0.016832,638.002163,4957.685439,27741.795385,1.623074,0.025146,0.009835,6.46,28.57,336.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0
2,20120524,US-Market,crm for financial,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000,14.647569,0.283518,0.016832,638.002163,4957.685439,27741.795385,1.623074,0.025146,0.009835,6.46,28.57,336.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0
3,20120524,US-Market,disaster recovery planning for it,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000,14.647569,0.283518,0.016832,638.002163,4957.685439,27741.795385,1.623074,0.025146,0.009835,6.46,28.57,336.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0
4,20120524,US-Market,tracking a vehicle,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000,14.647569,0.283518,0.016832,638.002163,4957.685439,27741.795385,1.623074,0.025146,0.009835,6.46,28.57,336.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0


# handling Text Data

In [ ]:
# Install Dependencies
!pip install stop-words

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp37-none-any.whl size=32917 sha256=b2981d4c069ef2fe5b0c6a026a44c8cff99891b9966a9c0977bedb6d16b14a90
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [ ]:
import nltk, re
from stop_words import get_stop_words
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
re.sub(r'[^a-zA-Z ]', "", 'mamaMsh38yd 8934bf , .. ..sd==][')

'mamaMshyd bf   sd'

In [ ]:
stop_words = get_stop_words('english')

In [ ]:
# Define Tokenization Function
def text_to_token(dataset, column):

    def my_tokenizer(s):
        s = s.lower()
        tokens = nltk.tokenize.word_tokenize(s)
        tokens = [t for t in tokens if len(t) > 2]
        tokens = [t for t in tokens if re.findall('[\x00-\x7FâäèéêëîïôœùûüÿçÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ]', t)]
        tokens = [t for t in tokens if t not in stop_words]
        return tokens

    tokenized = []

    word_index_map = {}

    # Iterate tokenization to all of data
    current_index = 0
    for row in dataset[column]:
        tokens = my_tokenizer(row)
        tokenized.append(tokens)
        for token in tokens:
            if token not in word_index_map:
                word_index_map[token] = current_index
                current_index += 1

    # Transform tokens into vector
    def tokens_to_vector(tokens):
        x = np.zeros(len(word_index_map) + 1)
        for t in tokens:
            i = word_index_map[t]
            x[i] += 1
        x = x / x.sum()
        return x

    # Create features from tokens
    N = len(dataset)
    features = np.zeros((N, len(word_index_map) + 1))
    i = 0
    for tokens in tokenized:
        x = tokens_to_vector(tokens)
        features[i,:] = x
        i += 1
    
    return pd.DataFrame(features)

In [ ]:
tokenized = text_to_token(dataset, 'Keyword')
tokenized

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793
0,0.333333,0.333333,0.333333,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
1,0.000000,0.000000,0.000000,0.333333,0.333333,0.333333,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.5,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.333333,0.333333,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346637,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,0.0
346638,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
346639,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
346640,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

# Imbalanced Data

In [ ]:
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
dataset.head()

,Date,Market,Keyword,Average.Position,Clicks,Impressions,Cost,CPI,CPC,CTR,mean_market_CPC,mean_market_CPI,mean_market_CTR,mean_market_Clicks,mean_market_Cost,mean_market_Impressions,median_market_CPC,median_market_CPI,median_market_CTR,median_market_Clicks,median_market_Cost,median_market_Impressions,max_market_CPC,max_market_CPI,max_market_CTR,max_market_Clicks,max_market_Cost,max_market_Impressions
0,20120524,US-Market,secure online back up,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000,14.647569,0.283518,0.016832,638.002163,4957.685439,27741.795385,1.623074,0.025146,0.009835,6.46,28.57,336.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0
1,20120524,US-Market,agile management software,1.0,21.22,260.0,25.45,0.097885,1.19934,0.081615,14.647569,0.283518,0.016832,638.002163,4957.685439,27741.795385,1.623074,0.025146,0.009835,6.46,28.57,336.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0
2,20120524,US-Market,crm for financial,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000,14.647569,0.283518,0.016832,638.002163,4957.685439,27741.795385,1.623074,0.025146,0.009835,6.46,28.57,336.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0
3,20120524,US-Market,disaster recovery planning for it,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000,14.647569,0.283518,0.016832,638.002163,4957.685439,27741.795385,1.623074,0.025146,0.009835,6.46,28.57,336.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0
4,20120524,US-Market,tracking a vehicle,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000,14.647569,0.283518,0.016832,638.002163,4957.685439,27741.795385,1.623074,0.025146,0.009835,6.46,28.57,336.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0


In [ ]:
dataset.loc[dataset['CTR'] < 0.02, 'labels'] = 0
dataset.loc[dataset['CTR'] >= 0.02, 'labels'] = 1
dataset['labels'].value_counts()

0.0    220850
1.0    125792
Name: labels, dtype: int64

In [ ]:
total = 220850 + 125792
print(125792/total, ':', 220850/total)

0.36288735929287275 : 0.6371126407071273


In [ ]:
dataset = dataset.drop(['Date', 'Market', 'Keyword'], axis=1)

In [ ]:
X = dataset.drop(['labels'], 1)
y = dataset['labels']

In [ ]:
X = pd.concat([X, tokenized], 1)

0         0.0
1         1.0
2         0.0
3         0.0
4         0.0
         ... 
346637    0.0
346638    0.0
346639    0.0
346640    0.0
346641    0.0
Name: labels, Length: 346642, dtype: float64

In [ ]:
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
pd.Series(y).value_counts()

1.0    220850
0.0    220850
dtype: int64

In [ ]:
220850 - 125792

95058